In [2]:
import pandas as pd
import numpy as np

import acquire_news
import prepare

import warnings
warnings.filterwarnings('ignore')

In [3]:
news = acquire_news.make_new_request()
news.head()

stuff
[{'title': "I'll take it on the chin: Cave explorer after losing case against Elon Musk", 'content': 'British cave explorer Vernon Unsworth, who lost the defamation case he filed against Elon Musk for calling him a "pedo guy", said he respects the jury\'s decision. "I’ll take it on the chin and move on," Unsworth added. His legal team sought at least $190 million in damages from the Tesla CEO for his tweets about the diver.', 'category': 'business', 'author': 'Krishna Veera Vanamali', 'published_date': '2019-12-07T15:31:16.000Z'}, {'title': 'My faith in humanity is restored: Musk after winning defamation case', 'content': 'After a US jury found that Elon Musk did not defame British cave explorer Vernon Unsworth by calling him a "pedo guy" on Twitter, the Tesla CEO said, "My faith in humanity is restored". The 48-year-old billionaire argued that he did not intend to call Unsworth a paedophile, but instead was using "pedo guy" to mean "creepy old guy".', 'category': 'business', 'au

,author,category,content,published_date,title
0,Krishna Veera Vanamali,business,"British cave explorer Vernon Unsworth, who los...",2019-12-07T15:31:16.000Z,I'll take it on the chin: Cave explorer after ...
1,Krishna Veera Vanamali,business,After a US jury found that Elon Musk did not d...,2019-12-07T16:34:59.000Z,My faith in humanity is restored: Musk after w...
2,Dharna,business,A Lucknow-based customer has filed an FIR agai...,2019-12-09T08:17:05.000Z,FIR filed against Club Factory in Lucknow for ...
3,Krishna Veera Vanamali,business,Congress leader Shashi Tharoor said he wants a...,2019-12-08T01:30:55.000Z,I want a 'New India' where Bajaj can speak fea...
4,Pragya Swastik,business,Price of onion has shot up to ₹200 per kg in B...,2019-12-08T08:44:31.000Z,Onion prices surge up to ₹200 per kg in Bengaluru


In [4]:
df = news[["content", "category"]]

In [5]:
df["stemmed"] = df.content.apply(prepare.basic_clean).apply(prepare.tokenize).apply(prepare.stem)
df["lemmatized"] = df.content.apply(prepare.basic_clean).apply(prepare.tokenize).apply(prepare.lemmatize)

In [6]:
df.head()

,content,category,stemmed,lemmatized
0,"British cave explorer Vernon Unsworth, who los...",business,british cave explor vernon unsworth who lost t...,british cave explorer vernon unsworth who lost...
1,After a US jury found that Elon Musk did not d...,business,after a us juri found that elon musk did not d...,after a u jury found that elon musk did not de...
2,A Lucknow-based customer has filed an FIR agai...,business,a lucknowbas custom ha file an fir against chi...,a lucknowbased customer ha filed an fir agains...
3,Congress leader Shashi Tharoor said he wants a...,business,congress leader shashi tharoor said he want a ...,congress leader shashi tharoor said he want a ...
4,Price of onion has shot up to ₹200 per kg in B...,business,price of onion ha shot up to 200 per kg in ben...,price of onion ha shot up to 200 per kg in ben...


### TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df.content)
y = df.category

### MODELING

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123,stratify=y)

In [22]:
dt = DecisionTreeClassifier(criterion="entropy",random_state=123, min_samples_split=5, min_samples_leaf=1)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [23]:
yhat = dt.predict(X_train)

In [35]:
conf_matrix = confusion_matrix(y_train, yhat)
categories = ["business","entertainment","sports","technology"]

In [36]:
pd.DataFrame(conf_matrix,index=categories, columns=categories)

,business,entertainment,sports,technology
business,18,0,0,1
entertainment,0,19,1,0
sports,0,2,18,0
technology,2,0,1,17


In [27]:
print(classification_report(y_train, yhat))

               precision    recall  f1-score   support

     business       0.90      0.95      0.92        19
entertainment       0.90      0.95      0.93        20
       sports       0.90      0.90      0.90        20
   technology       0.94      0.85      0.89        20

     accuracy                           0.91        79
    macro avg       0.91      0.91      0.91        79
 weighted avg       0.91      0.91      0.91        79

